In [1]:
pip install openai python-dotenv requests colorama


In [2]:
import os
import json
import time
import logging
from typing import List, Dict, Any, Optional
from datetime import datetime

# Required packages - install with pip:
# pip install openai python-dotenv requests colorama

import openai
import requests
from dotenv import load_dotenv
from colorama import Fore, Style, init

# Initialize colorama for cross-platform colored terminal output
init()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    filename='chatbot.log'
)
logger = logging.getLogger("Chatbot")

class Chatbot:
    """
    A customizable chatbot using OpenAI's GPT model with conversation history.
    Can be extended for specific domains like fashion, beauty, etc.
    """
    def __init__(
        self,
        api_key: Optional[str] = None,
        model: str = "gpt-3.5-turbo",
        system_message: str = "You are a helpful assistant.",
        temperature: float = 0.7,
        max_tokens: int = 150,
        max_history: int = 10
    ):
        """Initialize the chatbot with configuration parameters."""
        # Load environment variables from .env file if present
        load_dotenv()

        # Use API key from parameters, environment variable, or prompt user
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            self.api_key = input("Please enter your OpenAI API key: ")

        # Set OpenAI API key
        openai.api_key = self.api_key

        # Model configuration
        self.model = model
        self.system_message = system_message
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.max_history = max_history

        # Initialize conversation history with system message
        self.conversation_history = [{"role": "system", "content": system_message}]

        logger.info(f"Chatbot initialized with model: {model}")

    def get_response(self, user_input: str) -> str:
        """Get a response from the chatbot based on user input."""
        try:
            # Add user input to conversation history
            self.conversation_history.append({"role": "user", "content": user_input})

            # Limit conversation history to max_history
            if len(self.conversation_history) > self.max_history + 1:  # +1 for system message
                # Keep system message and last N messages
                self.conversation_history = [
                    self.conversation_history[0]
                ] + self.conversation_history[-(self.max_history):]

            # Make API call to OpenAI
            response = openai.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                temperature=self.temperature,
                max_tokens=self.max_tokens
            )

            # Extract response text
            response_text = response.choices[0].message.content.strip()

            # Add assistant response to conversation history
            self.conversation_history.append({"role": "assistant", "content": response_text})

            return response_text

        except Exception as e:
            logger.error(f"Error getting response: {str(e)}")
            return f"Sorry, I encountered an error: {str(e)}"

    def save_conversation(self, filename: str = None) -> None:
        """Save the conversation history to a JSON file."""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"conversation_{timestamp}.json"

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.conversation_history, f, indent=2, ensure_ascii=False)
            logger.info(f"Conversation saved to {filename}")
            print(f"Conversation saved to {filename}")
        except Exception as e:
            logger.error(f"Error saving conversation: {str(e)}")
            print(f"Error saving conversation: {str(e)}")

    def load_conversation(self, filename: str) -> bool:
        """Load conversation history from a JSON file."""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                self.conversation_history = json.load(f)
            logger.info(f"Conversation loaded from {filename}")
            print(f"Conversation loaded from {filename}")
            return True
        except Exception as e:
            logger.error(f"Error loading conversation: {str(e)}")
            print(f"Error loading conversation: {str(e)}")
            return False

    def clear_history(self) -> None:
        """Clear conversation history, keeping only the system message."""
        self.conversation_history = [{"role": "system", "content": self.system_message}]
        logger.info("Conversation history cleared")
        print("Conversation history cleared")


class FashionRecommendationChatbot(Chatbot):
    """
    A specialized chatbot for fashion and outfit recommendations.
    Extends the base Chatbot class with domain-specific functionality.
    """
    def __init__(self, api_key: Optional[str] = None):
        # Specialized system message for fashion recommendations
        system_message = """
        You are a fashion expert assistant who helps users find the perfect outfits
        and fashion items. Provide specific recommendations based on user preferences,
        body type, occasion, weather, and current fashion trends. When making
        recommendations, be specific about colors, styles, and types of garments.
        """

        super().__init__(
            api_key=api_key,
            system_message=system_message,
            temperature=0.8,  # Slightly higher for more creative recommendations
            max_tokens=200    # Longer responses for detailed recommendations
        )

        # Common fashion recommendation categories
        self.categories = {
            "casual": ["jeans", "t-shirts", "sneakers", "casual dresses", "sweaters"],
            "formal": ["suits", "dress shirts", "formal dresses", "heels", "ties"],
            "party": ["cocktail dresses", "blazers", "statement accessories", "heels"],
            "workwear": ["blouses", "slacks", "pencil skirts", "loafers", "blazers"]
        }

    def recommend_outfit(self, occasion: str, preferences: str) -> str:
        """Generate a specialized outfit recommendation."""
        prompt = f"Please recommend an outfit for a {occasion} occasion with these preferences: {preferences}"
        return self.get_response(prompt)


def run_chatbot_cli():
    """Run the chatbot in a command-line interface."""
    print(f"{Fore.CYAN}==============================")
    print("Welcome to the Python Chatbot!")
    print("Type 'exit' to quit, 'save' to save the conversation,")
    print("'load' to load a conversation, or 'clear' to clear history.")
    print(f"=============================={Style.RESET_ALL}\n")

    # Uncomment the specific type of chatbot you want to use
    chatbot = Chatbot()
    # chatbot = FashionRecommendationChatbot()

    while True:
        # Get user input
        user_input = input(f"{Fore.GREEN}You: {Style.RESET_ALL}")

        # Check for special commands
        if user_input.lower() == 'exit':
            print(f"{Fore.YELLOW}Goodbye!{Style.RESET_ALL}")
            break
        elif user_input.lower() == 'save':
            filename = input(f"{Fore.YELLOW}Enter filename (or press Enter for auto-generated): {Style.RESET_ALL}")
            chatbot.save_conversation(filename if filename else None)
            continue
        elif user_input.lower() == 'load':
            filename = input(f"{Fore.YELLOW}Enter filename to load: {Style.RESET_ALL}")
            chatbot.load_conversation(filename)
            continue
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            print(f"{Fore.YELLOW}Conversation history cleared.{Style.RESET_ALL}")
            continue

        # Get and display response
        print(f"{Fore.BLUE}Bot: {Style.RESET_ALL}", end="")

        # You can implement streaming response here for a more interactive feel
        response = chatbot.get_response(user_input)
        print(response)
        print()


# For web integration with Flask (uncomment to use)
"""
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)
chatbot = Chatbot()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_message = data.get('message', '')

    if not user_message:
        return jsonify({'error': 'No message provided'}), 400

    response = chatbot.get_response(user_message)
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(debug=True)
"""

# Run the CLI chatbot if executed directly
if __name__ == "__main__":
    run_chatbot_cli()


Welcome to the Python Chatbot!
Type 'exit' to quit, 'save' to save the conversation,
'load' to load a conversation, or 'clear' to clear history.

Please enter your OpenAI API key: 1234
You: hi
Bot: 

ERROR:Chatbot:Error getting response: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 1234. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Sorry, I encountered an error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 1234. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

You: hello
Bot: 

ERROR:Chatbot:Error getting response: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 1234. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Sorry, I encountered an error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 1234. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}



KeyboardInterrupt: Interrupted by user